In [ ]:
import os
import torch
import pandas as pd

from airllm import AutoModel

from huggingface_hub import snapshot_download
from huggingface_hub import notebook_login
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


#os.environ['TRANSFORMERS_CACHE'] = 'D:\\Disco\\Data\\models\\'


In [ ]:
print(torch.__version__)

torch.cuda.is_available()

In [ ]:
print(os.environ['HF_HOME'])

In [ ]:
HUGGING_FACE_API_KEY = '' #os.environ.get("HUGGING_FACE_API_KEY_BRANDO")
notebook_login()

In [ ]:
path = 'D:/Disco/Data/PUCRS/projeto CDIA II/'
models_path = 'D:/Disco/Data/'
MAX_LENGTH = 4096

#### Carrega os modelos

In [ ]:
pipe = pipeline("token-classification", model="d4data/biomedical-ner-all")

In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [ ]:
llm_model = AutoModel.from_pretrained('Qwen/Qwen-72B')
#AutoModel.from_pretrained("/home/ubuntu/.cache/huggingface/hub/models--garage-bAInd--Platypus2-70B-instruct/snapshots/b585e74bcaae02e52665d9ac6d23f4d0dbc81a0f")

In [ ]:
# prepare the input text
input_text = [
    'What is the capital of United States?',
]


In [ ]:

# tokenize the input text
input_tokens = llm_model.tokenizer(input_text,
    return_tensors="pt",
    return_attention_mask=False,
    truncation=True,
    max_length=MAX_LENGTH,
    padding=False)


In [ ]:

# generate the output text
generation_output = llm_model.generate(
    input_tokens['input_ids'].cuda(),
    max_new_tokens=MAX_LENGTH,
    use_cache=True,
    return_dict_in_generate=True)

# decode the output text
output = llm_model.tokenizer.decode(generation_output.sequences[0])

# print the output text
print(output)

In [ ]:
def translate_to_EN(text):
    tokenizer.src_lang = "pt_XX"
    encoded_text = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_text,
        forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
    )

    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    return translated_text

In [ ]:
cons_reg = pd.read_csv(f'{path}amostra-dados/t_cons_reg.csv', sep=';')

In [ ]:
#cria um arquivo para cada paciente contendo todos os registros de anotações das consultas realizadas
for npac in cons_reg['npac'].unique():
    text = '\n-----------------------\n\n'.join([t for t in cons_reg[cons_reg['npac'] == npac]['coment_cons_reg']])
    with open(f'{path}cons_regs/{npac}.txt', 'w') as txt_file:
        txt_file.write(text)

In [ ]:
cons_reg

In [ ]:
#extrai as entidades e retorna um df contendo informacoes sobre as entidades extraidas

def extract_entities(text):

    translated_text = translate_to_EN(text)
    print(translated_text)
    pd.DataFrame(pipe(translated_text))
    
    #faz a uniao das subpartes